# Calculate probabilistic impact yearset

This module generates a yearly impact `yimp` object which contains probabilistic annual impacts for a specified amount of years (`sampled_years`). The impact values are extracted from a given impact `imp` object that contains impact values per event. The amount of `sampled_years` can be specified as an integer or as a list of years to be sampled for. The amount of events per sampled year (`events_per_year`) are determined with a Poisson distribution centered around n_events per year (`lam` = sum(event_impacts.frequency). Then, the probabilistic events occuring in each sampled year are sampled uniformaly from the input `imp` object and summed up per year. Thus, the `yimp` object contains the sum of sampled (event) impacts for each sampled year. In contrast to the expected annual impact (eai), an `yimp` object contains an impact for EACH sampled year and this value differs among years. The number of events_per_year and the selected_events are saved in a sampling vector (`sampling_vect`). 

The function impact_yearsets performs all these computational steps, taking an `imp` and the number of sampled_years (`sampled_years`) as input. The output of the function is the `yimp` object and the `sampling_vect`.
Moreover, a `sampling_vect` (generated in a previous run) can be provided as optional input and the user can define `lam` and decide whether a correction factor shall be applied (the default is applying the correction factor).
Reapplying the same sampling_vect does not only allow to reproduce the generated `yimp`, but also for a physically consistent way of sampling impacts caused by different hazards. 
The correction factor that is applied when the optional input `correction_fac`= True is a scaling of the computed `yimp` that assures that the eai(`yimp`) = eai(`imp`).

To make the process more transparent, this tutorial shows the single computations that are performed when generating an `yimp` object for a dummy event_impacts object. 

In [5]:
import numpy as np

import climada.util.yearsets as yearsets
from climada.engine import Impact

# dummy event_impacts object containing 10 event_impacts with the values 10-110 
# and the frequency 0.2 (Return period of 5 years)
imp = Impact()
imp.at_event = np.arange(10,110,10)
imp.frequency = np.array(np.ones(10)*0.2)

# the number of years to sample impacts for (length(yimp.at_event) = sampled_years)
sampled_years = 10

# sample number of events per sampled year
lam = np.sum(imp.frequency)
events_per_year = yearsets.sample_n_events(sampled_years, imp, lam)

# generate the sampling vector
sampling_vect = yearsets.sample_events(events_per_year, imp.frequency)

# calculate the impact per year
imp_per_year = yearsets.compute_imp_per_year(imp, sampling_vect)

# calculate the correction factor
correction_factor = yearsets.calculate_correction_fac(imp_per_year, imp)


# compare the resulting yimp with our step-by-step computation without applying the correction factor: 
    
yimp, sampling_vect = yearsets.impact_yearset(imp, sampling_vect=sampling_vect,
                                                        correction_fac = False)

print('The yimp.at_event values equal our step-by-step computed imp_per_year:')
print('yimp.at_event = ', yimp.at_event)
print('imp_per_year = ', imp_per_year)


# and here the same comparison with applying the correction factor (default settings):
yimp, sampling_vect = yearsets.impact_yearset(imp, sampling_vect=sampling_vect)

print('The same can be shown for the case of applying the correction factor.' 
      'The yimp.at_event values equal our step-by-step computed imp_per year:')
print('yimp.at_event = ', yimp.at_event)
print('imp_per_year = ', imp_per_year/correction_factor)




2021-06-02 16:21:06,423 - climada.util.yearsets - INFO - The correction factor amounts to -14.72868217054264
The yimp.at_event values equal our step-by-step computed imp_per_year:
yimp.at_event =  [ 60.  70.  90. 260.   0. 270. 140.   0. 210. 190.]
imp_per_year =  [ 60.  70.  90. 260.   0. 270. 140.   0. 210. 190.]
2021-06-02 16:21:06,431 - climada.util.yearsets - INFO - The correction factor amounts to -14.72868217054264
The same can be shown for the case of applying the correction factor.The yimp.at_event values equal our step-by-step computed imp_per year:
yimp.at_event =  [ 70.36363636  82.09090909 105.54545455 304.90909091   0.
 316.63636364 164.18181818   0.         246.27272727 222.81818182]
imp_per_year =  [ 70.36363636  82.09090909 105.54545455 304.90909091   0.
 316.63636364 164.18181818   0.         246.27272727 222.81818182]
